env list

In [35]:
pip install ecmwflibs

476.92s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install eccodes

484.98s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [72]:
pip install eccodes --upgrade

1300.46s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [28]:
 #!/usr/bin/python3

import numpy as np
import math
import datetime
import sys
import os
import xarray as xr
import re
import cfgrib

# plot modules
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.pyplot as plt
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs

In [29]:
filename = '/home/jovyan/work/private/'

In [77]:
aap = cfgrib.open_dataset(filename+'HA40_N55_202401020600_01200_GB.grib', engine='cfgrib')

DatasetBuildError: multiple values for unique key, try re-open the file with one of:
    filter_by_keys={'typeOfLevel': 'heightAboveGround'}
    filter_by_keys={'typeOfLevel': 'heightAboveSea'}
    filter_by_keys={'typeOfLevel': 'entireAtmosphere'}
    filter_by_keys={'typeOfLevel': 'isobaricInhPa'}

In [82]:
aap = cfgrib.open_datasets(filename+'HA40_N55_202401020600_01200_GB.grib', engine='cfgrib',  filter_by_keys={'typeOfLevel': 'heightAboveSea', 'stepType': 'instant'})

In [83]:
aap

[<xarray.Dataset>
 Dimensions:           (y: 340, x: 340)
 Coordinates:
     time              datetime64[ns] 2024-01-02T06:00:00
     step              timedelta64[ns] 12:00:00
     entireAtmosphere  float64 0.0
     latitude          (y, x) float64 ...
     longitude         (y, x) float64 ...
     valid_time        datetime64[ns] ...
 Dimensions without coordinates: y, x
 Data variables:
     unknown           (y, x) float32 ...
 Attributes:
     GRIB_edition:            1
     GRIB_centre:             99
     GRIB_centreDescription:  DeBilt, Netherlands
     GRIB_subCentre:          0
     Conventions:             CF-1.7
     institution:             DeBilt, Netherlands,
 <xarray.Dataset>
 Dimensions:            (heightAboveGround: 7, y: 340, x: 340)
 Coordinates:
     time               datetime64[ns] 2024-01-02T06:00:00
     step               timedelta64[ns] 12:00:00
   * heightAboveGround  (heightAboveGround) float64 0.0 2.0 10.0 ... 200.0 300.0
     latitude           (y, x) f

In [66]:
aap = xr.open_dataset(filename+'HA40_N55_202401020600_01300_GB', engine='cfgrib', filter_by_keys={'typeOfLevel': 'heightAboveGround', 'stepType': 'instant', 'shortName': '3jklhkljhkl046'})

In [70]:
pip list

1253.47s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Package                       Version
----------------------------- -----------
affine                        2.4.0
alembic                       1.11.1
anyio                         3.6.2
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
asttokens                     2.2.1
async-generator               1.10
async-lru                     2.0.2
attrs                         23.1.0
automap                       0.6.2
Babel                         2.12.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.12.2
bleach                        6.0.0
blinker                       1.6.2
boltons                       23.0.0
branca                        0.6.0
Cartopy                       0.22.0
certifi                       2023.7.22
certipy                       0.1.3
cffi                          1.15.1
cfgrib                        0.9.10.4
cftime                        1.6.2
chars

Just some testing

In [ ]:
 #!/usr/bin/python3

import numpy as np
import math
import datetime
import sys
import os
import re

# plot modules
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.pyplot as plt
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs

# third party modules
import pygrib

''' Constants -------------------------------------------------------'''
# I/O
HOMEDIR = '/usr/people/zwagers/chronoplot/'
DATADIR = '/nobackup/users/zwagers/Dudley_Eunice/Harmonie2022021800/'
DATADIR = '/net/bxsdpsp/dpsp/production/harmonie/HA40_ens/mbr000/'
OUTDIR = DATADIR
OUTDIR = '/usr/people/zwagers/public_html/chronoplot/'
SHAPEFILE = ('/usr/people/zwagers/pyAVW/provinciegrenzen_2019/'
'2019_provinciegrenzen_kustlijn.shp')
PREFIX = 'HA40_N25_000_'
POSTFIX = '_GB'
MEMBER_PREFIX = 'mbr' 

# settings 
MINH = 1 # minimum fcst hour
MAXH = 48 # maximum fcst hour
STEP = 1 # step
TRESHOLD = 75 # 100 kph

# domain
LONN = 3.2
LONX = 7.5
LATN = 50.7
LATX = 53.6

# Harmonie parameter numbers
UGUST = 162
VGUST = 163

# other
KMH = 3.6
FIGSIZEX = 1800
FIGSIZEY = 1600 
DPI = 80

def plot(ax, LONN, LONX, LATN, LATX):
    """ Plot a basic background of the Netherlands
    
    in: ax instance and domain
    out: ax instance
    """
    
    # Plot map
    ax.set_extent([LONN, LONX, LATN, LATX])
    ax.set_aspect(aspect=1.5)
    ax.patch.set_facecolor('white')

    # get coastlines
    coast = shpreader.natural_earth(resolution='10m',
        category='physical',
        name='coastline')
    coastlines = shpreader.Reader(coast).geometries()

    # get country borders
    countries = shpreader.natural_earth(resolution='10m',
        category = 'cultural',
        name = 'admin_0_countries')
        
    # get Dutch border
    for country in shpreader.Reader(countries).records():
        if country.attributes['NAME'] == 'Netherlands':
            netherlands = country.geometry
            break
    else:
        raise ValueError('Unable to find the NLD boundary.') 
        
    # plot coastline without Dutch coastline
    coastlines_m_netherlands = [geom.difference(netherlands.buffer(0.001))
        for geom in coastlines]
    ax.add_geometries(coastlines_m_netherlands, ccrs.PlateCarree(),
        edgecolor='k', facecolor='none', linewidth=0.5, zorder=98)
        
    # plot provinces
    provs = shpreader.Reader(SHAPEFILE)
    for prov in provs.geometries():
        ax.add_geometries([prov], ccrs.PlateCarree(),
            edgecolor='k', facecolor='none', linewidth=0.5, zorder=99)
        
    return ax

def main():
    
    # create color map with 48 colors
    colmap = cm.get_cmap('tab20b', 20)
    cmaplist1 = [colmap(i) for i in range(colmap.N)]
    colmap = cm.get_cmap('tab20c', 20)
    cmaplist2 = [colmap(i) for i in range(colmap.N)]
    cmaplist = cmaplist1 + cmaplist2 + cmaplist1[0:8]
    # force the first color entry to be white
    cmaplist[0] = (1, 1, 1, 1.0)
    # create the new map
    colmap = LinearSegmentedColormap.from_list(
        'Custom cmap', cmaplist, 48)
    # percentages color map    
    colmap2 = cm.get_cmap('tab10', 10)
    cmaplist3 = [colmap2(i) for i in range(colmap2.N)]
    cmaplist3[0] = (1, 1, 1, 1.0)
    colmap2 = LinearSegmentedColormap.from_list(
        'Custom cmap', cmaplist3, 10)
    
    
    for t in range(MINH, MAXH+1, STEP):
        fcst = datetime.datetime.strptime(sys.argv[1], '%Y%m%d%H%M')
        + datetime.timedelta(hours=t)
        hour = "%05d" % (t*100)
        run = DATADIR + PREFIX + sys.argv[1] + '_' + hour + POSTFIX
        print (run)
        
        gr = pygrib.open(run)
        msgU = gr.select(indicatorOfParameter=UGUST)
        msgV = gr.select(indicatorOfParameter=VGUST)
        qnt = np.sqrt(msgU[0].values**2+msgV[0].values**2)*KMH
        
        if t==MINH:
            chrono_total = np.zeros_like(msgU[0].values)
            chrono_last = np.zeros_like(msgU[0].values)
        chrono_last = np.where(qnt>=TRESHOLD, t, chrono_last)
        chrono_total += np.where(qnt>=TRESHOLD, 1, 0)
        
    for t in range(MAXH, MINH-1, -1*STEP):
        fcst = datetime.datetime.strptime(sys.argv[1], '%Y%m%d%H%M')
        + datetime.timedelta(hours=t)
        hour = "%05d" % (t*100)
        run = DATADIR + PREFIX + sys.argv[1] + '_' + hour + POSTFIX
        print (run)
        
        gr = pygrib.open(run)
        msgU = gr.select(indicatorOfParameter=UGUST)
        msgV = gr.select(indicatorOfParameter=VGUST)
        qnt = np.sqrt(msgU[0].values**2+msgV[0].values**2)*KMH
        
        if t==MAXH:
            chrono_first = np.zeros_like(msgU[0].values)
            lats, lons = msgU[0].latlons() 
        chrono_first = np.where(qnt>=TRESHOLD, t, chrono_first)
        
    fig, ax = plt.subplots(
        ncols=2, nrows=2,
        figsize=(FIGSIZEX/DPI, FIGSIZEY/DPI), 
        dpi=DPI,
        subplot_kw={'projection':ccrs.PlateCarree()}
        )

    ax[0][0] = plot(ax[0][0], LONN, LONX, LATN, LATX)
    ax[0][1] = plot(ax[0][1], LONN, LONX, LATN, LATX)
    ax[1][0] = plot(ax[1][0], LONN, LONX, LATN, LATX)
    ax[1][1] = plot(ax[1][1], LONN, LONX, LATN, LATX)

    plt.sca(ax[0][0])
    plt.pcolormesh(lons, lats, chrono_first, cmap=colmap, vmin=MINH-1, 
    vmax=MAXH)
    cbar = plt.colorbar(orientation='vertical')
    cbar.set_ticks(list(range(MINH-1, MAXH+1)))

    labels = ['']
    for t in range(MINH, MAXH+1):
        fcst = datetime.datetime.strptime(sys.argv[1], '%Y%m%d%H%M')\
        + datetime.timedelta(hours=t)
        labels.append(fcst.strftime('%a %d/%HZ'))
    cbar.set_ticklabels(labels)

    plt.title(f"Eerste keer optreden windstoot ≥ {TRESHOLD} km/u in run "+sys.argv[1])

    plt.sca(ax[0][1])
    plt.pcolormesh(lons ,lats, chrono_last, cmap=colmap, vmin=0, vmax=48)
    cbar = plt.colorbar(orientation='vertical')
    cbar.set_ticks(list(range(MINH-1, MAXH+1)))

    labels = ['']
    for t in range(MINH, MAXH+1):
        fcst = datetime.datetime.strptime(sys.argv[1], '%Y%m%d%H%M')\
        + datetime.timedelta(hours=t)
        labels.append(fcst.strftime('%a %d/%HZ'))
    cbar.set_ticklabels(labels)

    plt.title(f"Laatste keer optreden windstoot ≥ {TRESHOLD} km/u in run "+sys.argv[1])

    plt.sca(ax[1][0])
    plt.pcolormesh(lons, lats, chrono_last-chrono_first, 
    cmap=colmap, vmin=0, vmax=48)
    cbar = plt.colorbar(orientation='vertical')
    cbar.set_ticks(list(range(MINH-1, MAXH+1)))

    plt.title("Verschil tussen eerste en laatste (uren) in run "+sys.argv[1])


    plt.sca(ax[1][1])
    plt.pcolormesh(lons, lats,\
    100*(chrono_last-chrono_first)/(chrono_total-1),\
    cmap=colmap2, vmin=0, vmax=100)
    cbar = plt.colorbar(orientation='vertical')
    cbar.set_ticks(list(range(0, 100+1,10)))

    plt.title(\
    "Percentage uurvakken met ≥ 100 km/u tussen eerste en laatste "\
    +sys.argv[1])

    plt.show()

if __name__ == "__main__":
    main()


